# Spotify Streaming History Dashboard
## Part 3: Dashboard
Summary of process:
- Prepare various individual dataframes with different grouping properties (by day number, by streaming session number, by month, by day of week, by hour of day, and by day/session for top artist)
- Create dashboard layout using Dash
- Populate dashboard with Plotly graphics
***
### Install and Import Required Libraries

In [1]:
!pip install dash
!pip install plotly
!pip install jupyter-dash

In [2]:
import pandas as pd
import numpy as np
import dash
import math
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

### Set Up Filtered Dataframes for Specific Plots

In [3]:
df = pd.read_csv('StreamingHistory_Final.csv')

In [4]:
# GROUP BY DAY NUMBER (to get average audio feature values per day)
df_daynum = df.groupby(['day']).aggregate(
    {'msPlayed':np.sum, 
     'acousticness':np.average, 
     'danceability':np.average, 
     'energy':np.average, 
     'instrumentalness':np.average, 
     'liveness':np.average, 
     'loudness':np.average,
     'loudness_norm':np.average,
     'speechiness':np.average, 
     'tempo':np.average,
     'tempo_norm':np.average,
     'valence':np.average})
df_daynum['day'] = df_daynum.index

# Convert ms played to mins playes
df_daynum.rename({'msPlayed': 'minsPlayed'}, axis='columns', inplace=True)
df_daynum['minsPlayed'] = df_daynum['minsPlayed'].div(60000)
df_daynum.reset_index(inplace=True, drop=True)


# GROUP BY STREAMING SESSION (to get average audio feature values per streaming session)
df_session = df.groupby(['listeningSession']).aggregate(
    {'msPlayed':np.sum, 
     'acousticness':np.average, 
     'danceability':np.average, 
     'energy':np.average, 
     'instrumentalness':np.average, 
     'liveness':np.average, 
     'loudness':np.average,
     'loudness_norm':np.average,
     'speechiness':np.average, 
     'tempo':np.average,
     'tempo_norm':np.average,
     'valence':np.average})
df_session['listeningSession'] = df_session.index

# Convert ms played to mins played
df_session.rename({'msPlayed': 'minsPlayed'}, axis='columns', inplace=True)
df_session['minsPlayed'] = df_session['minsPlayed'].div(60000)
df_session.reset_index(inplace=True, drop=True)


# GROUP BY MONTH (to get average audio feature values per month)
df_month = df.groupby(['month']).aggregate(
    {'msPlayed':np.sum, 
     'acousticness':np.average, 
     'danceability':np.average, 
     'energy':np.average, 
     'instrumentalness':np.average, 
     'liveness':np.average, 
     'loudness':np.average,
     'loudness_norm':np.average,
     'speechiness':np.average, 
     'tempo':np.average,
     'tempo_norm':np.average,
     'valence':np.average})
df_month['month'] = df_month.index

# Convert ms played to hrs played
df_month.rename({'msPlayed': 'hrsPlayed'}, axis='columns', inplace=True)
df_month['hrsPlayed'] = df_month['hrsPlayed'].div(3600000)
df_month.reset_index(inplace=True, drop=True)

# Re-index so months are in order
df_month = df_month.reindex([4, 3, 7, 0, 8, 6, 5, 1, 11, 10, 9, 2])


# GROUP BY DAY OF WEEK (to get average audio feature values per day of the week)
df_dayofweek = df.groupby(['dayofweek']).aggregate(
    {'msPlayed':np.sum, 
     'acousticness':np.average, 
     'danceability':np.average, 
     'energy':np.average, 
     'instrumentalness':np.average, 
     'liveness':np.average, 
     'loudness':np.average,
     'loudness_norm':np.average,
     'speechiness':np.average, 
     'tempo':np.average,
     'tempo_norm':np.average,
     'valence':np.average})
df_dayofweek['dayofweek'] = df_dayofweek.index

# Convert ms played to hrs played
df_dayofweek.rename({'msPlayed': 'hrsPlayed'}, axis='columns', inplace=True)
df_dayofweek['hrsPlayed'] = df_dayofweek['hrsPlayed'].div(3600000)
df_dayofweek.reset_index(inplace=True, drop=True)

# Re-index so days of week are in order
df_dayofweek = df_dayofweek.reindex([1, 5, 6, 4, 0, 2, 3])


# GROUP BY HOUR OF DAY (to get average audio feature values per hour of day)
df_hour = df.groupby(['hour']).aggregate(
    {'msPlayed':np.sum, 
     'acousticness':np.average, 
     'danceability':np.average, 
     'energy':np.average, 
     'instrumentalness':np.average, 
     'liveness':np.average, 
     'loudness':np.average,
     'loudness_norm':np.average,
     'speechiness':np.average, 
     'tempo':np.average,
     'tempo_norm':np.average,
     'valence':np.average})
df_hour['hour'] = df_hour.index
# Convert ms played to hrs played
df_hour.rename({'msPlayed': 'hrsPlayed'}, axis='columns', inplace=True)
df_hour['hrsPlayed'] = df_hour['hrsPlayed'].div(3600000)
df_hour.reset_index(inplace=True, drop=True)


# FIND TOP ARTIST 
top_artist = df.groupby('artistName').sum().sort_values('msPlayed', ascending=False).index[0]

# Add columns to show if top artist was played and if so, for how long
topArtist = []
topArtist_ms = []

for index, row in enumerate(df.iterrows()):
    if row[1]['artistName']==top_artist:
        topArtist.append(1)
        topArtist_ms.append(row[1]['msPlayed'])
    else:
        topArtist.append(0)
        topArtist_ms.append(0)
        
df['topArtist'] = topArtist
df['topArtist_ms'] = topArtist_ms


# TOP ARTIST - Grouped by day
df_topartist_daynum = df.groupby(['day']).aggregate(
    {'msPlayed':np.sum, 
     'topArtist':np.sum, 
     'topArtist_ms':np.sum})
df_topartist_daynum['day'] = df_topartist_daynum.index

# Convert ms played to mins played
df_topartist_daynum.rename({'msPlayed': 'minsPlayed', 'topArtist_ms': 'topArtist_mins'}, axis='columns', inplace=True)
df_topartist_daynum['minsPlayed'] = df_topartist_daynum['minsPlayed'].div(60000)
df_topartist_daynum['topArtist_mins'] = df_topartist_daynum['topArtist_mins'].div(60000)
df_topartist_daynum.reset_index(inplace=True, drop=True)

# Add percent of top artist streamed relative to total streaming time
df_topartist_daynum['topArtist_percPlayed'] = df_topartist_daynum['topArtist_mins'] / df_topartist_daynum['minsPlayed'] * 100
df_topartist_daynum.loc[df_topartist_daynum['topArtist'] == 0, 'topArtist_percPlayed'] = 100

# Change topArtist column to Booleans
df_topartist_daynum['topArtist'] = df_topartist_daynum['topArtist']>0
df_topartist_daynum.drop(df_topartist_daynum[df_topartist_daynum.day>364].index, inplace=True)

# Add x and y coordinates for scatterplot
n = 28
x_coord = []
y_coord = []
for index, row in enumerate(df_topartist_daynum.iterrows()):
    xcoord = row[1]['day']%n
    if xcoord == 0:
        xcoord=n
    x_coord.append(xcoord)
    y_coord.append(math.floor(row[1]['day']/-n))

df_topartist_daynum['x_coord'] = x_coord
df_topartist_daynum['y_coord'] = y_coord


# TOP ARTIST - Grouped by streaming session
df_topartist_session = df.groupby(['listeningSession']).aggregate(
    {'msPlayed':np.sum, 
     'topArtist':np.sum, 
     'topArtist_ms':np.sum})
df_topartist_session['listeningSession'] = df_topartist_session.index

# Convert ms played to mins played
df_topartist_session.rename({'msPlayed': 'minsPlayed', 'topArtist_ms': 'topArtist_mins'}, axis='columns', inplace=True)
df_topartist_session['minsPlayed'] = df_topartist_session['minsPlayed'].div(60000)
df_topartist_session['topArtist_mins'] = df_topartist_session['topArtist_mins'].div(60000)
df_topartist_session.reset_index(inplace=True, drop=True)

# Add percent of top artist streamed relative to total streaming time
df_topartist_session['topArtist_percPlayed'] = df_topartist_session['topArtist_mins'] / df_topartist_session['minsPlayed'] * 100
df_topartist_session.loc[df_topartist_session['topArtist'] == 0, 'topArtist_percPlayed'] = 100

# Change topArtist column to Booleans
df_topartist_session['topArtist'] = df_topartist_session['topArtist']>0
df_topartist_session.drop(df_topartist_session[df_topartist_session.listeningSession>1056].index, inplace=True)

# Add x and y coordinates for scatterplot
n = 44
x_coord = []
y_coord = []
for index, row in enumerate(df_topartist_session.iterrows()):
    xcoord = row[1]['listeningSession']%n
    if xcoord == 0:
        xcoord=n
    x_coord.append(xcoord)
    y_coord.append(math.floor(row[1]['listeningSession']/-n))

df_topartist_session['x_coord'] = x_coord
df_topartist_session['y_coord'] = y_coord

### Create Dashboard Layout and Populate with Interactive Plotly Plots

In [5]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[html.Br(),
                                html.H1('Spotify Streaming History Dashboard',
                                        style={'textAlign': 'center', 'font-size': 40, 'font-family': 'helvetica'}),
                                html.Br(), 
                                # PART 1 - Streaming Habits Insights
                                html.Div(children=[
                                    html.H2('Part 1: Streaming Habits Insights',
                                        style={'textAlign': 'center', 'font-size': 30}),
                                    html.Center(children=[
                                        # DROPDOWN 1 - Time interval
                                        html.Div(children='Select Time Interval', 
                                                 style={'font-style': 'italic', 'font-weight': 'bold'}),
                                        dcc.Dropdown(id='interval-dropdown-1', 
                                                     options=[{'label': 'Day', 'value': 'per_day'},
                                                              {'label':'Streaming Session', 'value': 'per_session'}], 
                                                     placeholder='Select Interval', 
                                                     value='per_day',
                                                     searchable=True,
                                                     style={'textAlign': 'left', 'width':'200px', 'display': 'inline-block'}),
                                        html.Br(),
                                        html.Br(),
                                        # DROPDOWN 2 - Audio Feature
                                        html.Div(children='Select Audio Feature', 
                                                 style={'font-style': 'italic', 'font-weight': 'bold'}),
                                        dcc.Dropdown(id='feature-dropdown-1', 
                                                     options=[{'label': 'Acousticness', 'value': 'acousticness'},
                                                              {'label': 'Danceability', 'value': 'danceability'},
                                                              {'label': 'Energy', 'value': 'energy'},
                                                              {'label': 'Instrumentalness', 'value': 'instrumentalness'},
                                                              {'label': 'Liveness', 'value': 'liveness'},
                                                              {'label': 'Loudness', 'value': 'loudness'},
                                                              {'label': 'Speechiness', 'value': 'speechiness'},
                                                              {'label': 'Tempo', 'value': 'tempo'},
                                                              {'label': 'Valence', 'value': 'valence'}], 
                                                     placeholder='Select Feature',
                                                     value='valence',
                                                     searchable=True,
                                                     style={'textAlign':'left', 'width':'200px', 'display': 'inline-block'})]),
                                    html.Br(),
                                    # PLOT 1 - Streaming by Month
                                    dcc.Graph(id='month-bar-chart',
                                              style={'width':'44%', 'display': 'inline-block', 'margin-right': '1.8%', 'margin-left': '4%', 'border': '1px green solid'},
                                              figure={'layout':{'paper_bgcolor': '#66FF66'}}),
                                    # PLOT 2 - Streaming by Day of Week
                                    dcc.Graph(id='day-bar-chart',
                                              style={'width':'44%', 'display': 'inline-block', 'margin-right': '4%', 'margin-left': '1.8%', 'border': '1px green solid'},
                                              figure={'layout':{'paper_bgcolor': '#66FF66'}}),
                                    html.Br(),
                                    html.Br(),
                                    # PLOT 3 - Streaming by Hour of Day
                                    dcc.Graph(id='hour-bar-chart',
                                              style={'width':'44%', 'display': 'inline-block', 'margin-right': '1.8%', 'margin-left': '4%', 'border': '1px green solid'},
                                              figure={'layout':{'paper_bgcolor': '#66FF66'}}),
                                    # PLOT 4 - All Audio Features (per day or per session)
                                    dcc.Graph(id='mood-heatmap',
                                                  figure={'layout':{'paper_bgcolor': '#99FF99'}},
                                                  style={'width': '44%', 'display': 'inline-block', 'margin-right': '4%', 'margin-left': '1.8%', 'border': '1px green solid'}), 
                                    html.P(' ')],
                                        style={'border': '2px black solid', 'backgroundColor': '#FFFFFF', 'width': '95%', 'margin': 'auto', 'font-family': 'helvetica'}),
                                html.Br(),
                                # PART 2 - Personalized Visualizations
                                html.Div(children=[
                                    html.H2('Part 2: Personalized Visualizations',
                                            style={'textAlign': 'center', 'font-size': 30}),
                                    html.Center(children=[
                                        # DROPDOWN 3 - Time Interval
                                        html.Div(children='Select Time Interval', 
                                                 style={'font-style': 'italic', 'font-weight': 'bold'}),
                                        dcc.Dropdown(id='interval-dropdown-2', 
                                                     options=[{'label': 'Day', 'value': 'per_day'},
                                                              {'label':'Streaming Session', 'value': 'per_session'}], 
                                                     placeholder='Select Interval', 
                                                     value='per_day',
                                                     searchable=True,
                                                     style={'textAlign': 'left', 'width':'200px', 'display': 'inline-block'}),
                                        html.Br(),
                                        html.Br(),
                                        # DROPDOWN 4 - Audio Feature
                                        html.Div(children='Select Audio Feature', 
                                                 style={'font-style': 'italic', 'font-weight': 'bold'}),
                                        dcc.Dropdown(id='feature-dropdown-2', 
                                                     options=[{'label': 'Acousticness', 'value': 'acousticness'},
                                                              {'label': 'Danceability', 'value': 'danceability'},
                                                              {'label': 'Energy', 'value': 'energy'},
                                                              {'label': 'Instrumentalness', 'value': 'instrumentalness'},
                                                              {'label': 'Liveness', 'value': 'liveness'},
                                                              {'label': 'Loudness', 'value': 'loudness'},
                                                              {'label': 'Speechiness', 'value': 'speechiness'},
                                                              {'label': 'Tempo', 'value': 'tempo'},
                                                              {'label': 'Valence', 'value': 'valence'}], 
                                                     placeholder='Select Feature', 
                                                     value='valence',
                                                     searchable=True,
                                                     style={'textAlign':'left', 'width':'200px', 'display': 'inline-block'})]),
                                    html.Br(),
                                    # PLOT 5 - Audio Feature by Day or by Session
                                    dcc.Graph(id='daily-mood-bar-chart',
                                              figure={'layout':{'paper_bgcolor': '#99FF99'}},
                                              style={'width': '92%', 'border': '1px green solid', 'margin': 'auto'}),
                                    html.Br(),
                                    # PLOT 6 - Top Artist Appreciation
                                    dcc.Graph(id='top-artist-scatterplot', 
                                              figure={'layout':{'paper_bgcolor': '#CCFFCC'}},
                                              style={'width': '92%', 'border': '1px green solid', 'margin': 'auto'}),
                                    html.Br()],
                                         style={'border': '2px black solid', 'backgroundColor': '#FFFFFF', 'width': '95%', 'margin': 'auto', 'font-family': 'helvetica'}),
                                html.Br()],
                     style={'backgroundColor': '#ff8f40'})

# PLOT 1 - Streaming by Month
@app.callback(Output(component_id='month-bar-chart', component_property='figure'), 
              Input(component_id='feature-dropdown-1', component_property='value'))

def get_month_chart(entered_feature):
    fig = px.bar(df_month, 
                 x='month', 
                 y='hrsPlayed', 
                 color=entered_feature, 
                 labels={'hrsPlayed':'Total Streaming Time (hours)'})
    fig.update_traces(marker=dict(line=dict(width=0)))
    fig.update_layout(title='<b>Streaming Habits by Month</b>', 
                      title_x=0.5,
                      font_color='black',
                      plot_bgcolor='rgba(255, 255, 255, 0)', 
                      paper_bgcolor='rgba(255, 255, 255, 0)')
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_xaxes(title_text=' ')
    fig.update_coloraxes(colorbar=dict(title=str(entered_feature).capitalize(), thickness=15, tickformat='.2f'))
    return fig


# PLOT 2 - Streaming by Day of Week
@app.callback(Output(component_id='day-bar-chart', component_property='figure'), 
              Input(component_id='feature-dropdown-1', component_property='value'))

def get_dayofweek_chart(entered_feature):
    fig = px.bar(df_dayofweek, 
                 x='dayofweek', 
                 y='hrsPlayed', 
                 color=entered_feature, 
                 labels={'hrsPlayed':'Total Streaming Time (hours)'})
    fig.update_traces(marker=dict(line=dict(width=0)))
    fig.update_layout(title='<b>Streaming Habits by Day</b>',
                      title_x=0.5,
                      font_color='black',
                      plot_bgcolor='rgba(255, 255, 255, 0)', 
                      paper_bgcolor='rgba(255, 255, 255, 0)')
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_xaxes(title_text=' ')
    fig.update_coloraxes(colorbar=dict(title=str(entered_feature).capitalize(), thickness=15, tickformat='.2f'))
    return fig


# PLOT 3 - Streaming by Hour of Day
@app.callback(Output(component_id='hour-bar-chart', component_property='figure'), 
              Input(component_id='feature-dropdown-1', component_property='value'))

def get_hour_chart(entered_feature):
    fig = px.bar(df_hour, 
                 x='hour', 
                 y='hrsPlayed', 
                 color=entered_feature, 
                 labels={'hrsPlayed':'Total Streaming Time (hours)'})
    fig.update_traces(marker=dict(line=dict(width=0)))
    fig.update_layout(title='<b>Streaming Habits by Hour</b>', 
                      title_x=0.5, 
                      font_color='black',
                      plot_bgcolor='rgba(255, 255, 255, 0)', 
                      paper_bgcolor='rgba(255, 255, 255, 0)')
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_xaxes(title_text='Hour')
    fig.update_coloraxes(colorbar=dict(title=str(entered_feature).capitalize(), thickness=15, tickformat='.2f'))
    return fig


# PLOT 4 - All Audio Features (per day or per session)
@app.callback(Output(component_id='mood-heatmap', component_property='figure'), 
              Input(component_id='interval-dropdown-1', component_property='value'))

def get_heatmap(entered_interval):
    if entered_interval=='per_day':
        fig = px.imshow(df_daynum[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                                   'loudness_norm', 'speechiness', 'tempo_norm', 'valence']], 
                        labels=dict(x='Audio Feature', y='Day'), 
                        x=['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 
                           'Loudness', 'Speechiness', 'Tempo', 'Valence'])
        fig.update_layout(title='<b>Audio Features per Day Over Previous Year</b>', 
                          title_x=0.5, 
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)')        
        fig.update_yaxes(autorange=True) 
        fig.update_xaxes(title_text=' ')
        fig.update_coloraxes(colorbar=dict(thickness=15, tickformat='.2f'))
    else:
        fig = px.imshow(df_session[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                                    'loudness_norm', 'speechiness', 'tempo_norm', 'valence']],
                        labels=dict(x='Audio Feature', y='Streaming Session'), 
                        x=['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 
                           'Loudness', 'Speechiness', 'Tempo', 'Valence'])
        fig.update_layout(title='<b>Audio Features per Listening Session Over Previous Year</b>', 
                          title_x=0.5, 
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)')                
        fig.update_yaxes(autorange=True) 
        fig.update_xaxes(title_text=' ')
        fig.update_coloraxes(colorbar=dict(thickness=15, tickformat='.2f'))
    return fig


# PLOT 5 - Audio Feature by Day or by Session
@app.callback(Output(component_id='daily-mood-bar-chart', component_property='figure'), 
              [Input(component_id='interval-dropdown-2', component_property='value'), 
               Input(component_id='feature-dropdown-2', component_property='value')])

def get_mood_chart(entered_interval, entered_feature):
    if entered_interval=='per_day':
        fig = px.bar(df_daynum, 
                     x='day', 
                     y='minsPlayed', 
                     color=entered_feature,
                     labels={'minsPlayed': 'Total Streaming Time (minutes)', 'day': 'Day'})
        fig.update_traces(marker=dict(line=dict(width=0)))
        fig.update_layout(title='<b>Daily Streaming Time and Average ' + str(entered_feature).capitalize() + '</b>',
                          title_x=0.5,
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)', 
                          bargap=0)
        fig.update_yaxes(showgrid=False, zeroline=False)
        fig.update_xaxes(title_text='Day')
        fig.update_coloraxes(colorbar=dict(title=str(entered_feature).capitalize(), thickness=15, tickformat='.2f'))
    else:
        fig = px.bar(df_session, 
                     x='listeningSession', 
                     y='minsPlayed', 
                     color=entered_feature, 
                     labels={'minsPlayed':'Total Streaming Time (minutes)', 'listeningSession': 'Streaming Session'})
        fig.update_traces(marker=dict(line=dict(width=0)))
        fig.update_layout(title='<b>Streaming Session Time and Average ' + str(entered_feature).capitalize() + '</b>',
                          title_x=0.5,
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)', 
                          bargap=0)
        fig.update_yaxes(showgrid=False, zeroline=False)
        fig.update_xaxes(title_text='Streaming Session')
        fig.update_coloraxes(colorbar=dict(title=str(entered_feature).capitalize(), thickness=15, tickformat='.2f'))
    return fig


# PLOT 6 - Top Artist Appreciation
@app.callback(Output(component_id='top-artist-scatterplot', component_property='figure'), 
              Input(component_id='interval-dropdown-2', component_property='value'))

def get_artist_scatterplot(entered_interval):
    if entered_interval=='per_day':
        fig = px.scatter(df_topartist_daynum,
                         x='x_coord', 
                         y='y_coord', 
                         color='topArtist',
                         color_discrete_sequence=['orange', 'blue'],
                         symbol=df_topartist_daynum['topArtist'],
                         symbol_sequence=['circle', 'circle'], 
                         hover_data=['day'])
        fig.update_traces(marker=dict(size=15, line=dict(width=0)))        
        fig.update_yaxes(showgrid=False, visible=False)
        fig.update_xaxes(showgrid=False, visible=False)
        fig.update_layout(title='<b>Top Artist Appreciation by Day</b>', 
                          title_x=0.5, 
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)', 
                          legend=dict(title_text='Top Artist Played?', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))        
    else:
        fig = px.scatter(df_topartist_session, 
                         x='x_coord', 
                         y='y_coord', 
                         color='topArtist', 
                         color_discrete_sequence=['blue', 'orange'],
                         symbol=df_topartist_session['topArtist'],
                         symbol_sequence=['circle', 'circle'])
        fig.update_traces(marker=dict(size=10, line=dict(width=0)))        
        fig.update_yaxes(showgrid=False, visible=False)
        fig.update_xaxes(showgrid=False, visible=False)
        fig.update_layout(title='<b>Top Artist Appreciation by Streaming Session</b>', 
                          title_x=0.5, 
                          font_color='black',
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)', 
                          legend=dict(title_text='Top Artist Played?', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1, traceorder='reversed'))    
    return fig


# Run Dashboard
if __name__ == '__main__':
    app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


### Extra Plots Not Used

In [ ]:
# PLOT - Animated Radar
@app.callback(Output(component_id='mood-radar', component_property='figure'), 
              Input(component_id='interval-dropdown2', component_property='value'))

def get_radar(entered_interval):
    df_daynum_radar = df_daynum.drop(columns=['minsPlayed', 'loudness', 'tempo'])
    df_daynum_radar = pd.melt(df_daynum_radar, id_vars='day', var_name='feature', value_name='feature_value')
    df_session_radar = df_session.drop(columns=['minsPlayed', 'loudness', 'tempo'])
    df_session_radar = pd.melt(df_session_radar, id_vars='listeningSession', var_name='feature', value_name='feature_value')
    if entered_interval=='per_day':
        fig = px.line_polar(df_daynum_radar, 
                            r='feature_value', 
                            theta='feature', 
                            line_close=True, 
                            animation_frame='day', 
                            range_r=[0,1])
        fig.update_layout(title='Audio Features per Day Over Previous Year', 
                          title_x=0.5, 
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)') 
        fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1
        fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
        fig.update_geos(resolution=50)
    else:
        fig = px.line_polar(df_session_radar, 
                            r='feature_value', 
                            theta='feature', 
                            line_close=True, 
                            animation_frame='listeningSession', 
                            range_r=[0,1])
        fig.update_layout(title='Audio Features per Listening Session Over Previous Year', 
                          title_x=0.5, 
                          plot_bgcolor='rgba(255, 255, 255, 0)', 
                          paper_bgcolor='rgba(255, 255, 255, 0)') 
        fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1
        fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
        fig.update_geos(resolution=50)
    return fig
